# Get started with Azure Cosmos DB using the .NET SDK for SQL API

In this notebook, we'll learn how to do operations against the Azure Cosmos DB SQL API using the .NET SDK client library. We'll use an IOT scenario of vehicle telemetry to illustrate how to create resources, insert data, query and visualize our data. 

<br>To run this notebook, be sure to select the **CSharp** kernel in the command bar, so we can get all the language support features we need.

### Create database and containers
First, we'll create a new database and container to hold our data. Note when we create the container, we select a value for throughput (RU/s) and partition key. 
- Throughput: we'll start with 400 RU/s, the minimum RU/s for any Azure Cosmos DB resource.  
- Partition key:  we'll partition our data by Vehicle Identification Number (VIN), as it has a high cardinality (important for workloads during a lot of writes, e.g. IOT workloads) and evenly distributes the request and storage volume. Choosing a good partition key is "key" to getting good scale and performance from Azure Cosmos DB, so it's important we follow the [best practices](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview)!



In [ ]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and container with 400 RU/s
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("CosmosIOTDemo");
Container container = await database.CreateContainerIfNotExistsAsync("VehicleData", "/Vin", 400);

Display.AsMarkdown(@"
Created database CosmosIOTDemo and container VehicleData. You can see these new resources by refreshing your resource pane under the Data section.
");

### Create C# classes to represent our TelemetryEvent
We can create classes to represent the items we will be inserting into our container. In this sample, we'll be working with vehicle telemetry data that contains sensor information.

In [ ]:
using System;

class TelemetryEvent
{
    public string id { get; set; }
    
    public string Vin { get; set; }

    public string EventName { get; set; }

    public string VehicleType { get; set; }
        
    public string FuelType {get; set; }    

    public double s1 { get; set; }

    public double s2 { get; set; }

    public double s3 { get; set; }

    public DateTime timestamp { get; set; }

    public string date { get; set; }

    public string region { get; set; }

}

### Insert data into our container
First, we'll write a helper method we can call that randomly generates a set number of events that we will insert into our container. We'll use the [Bogus](https://github.com/bchavez/Bogus) library for .NET to easily generate sample data.

In [ ]:
#r "nuget: Bogus"
using System.Collections.Generic;
using System.Linq.Expressions;
using System;
using Bogus;

static List<TelemetryEvent> GenerateTelemetryEvents(int numberOfEventsToGenerate)
{
    var eventTypes = new string[] { "Harsh_break", "Airbag_deploy", "Check_engine_light" };

    var faker = new Faker();
    
    var vinList = Enumerable.Range(1, 2000)
    .Select(_ => _.ToString())
    .ToList();
    
    var telemetryEvent = new Faker<TelemetryEvent>()
    .StrictMode(true)
    //Generate telemetry event
    .RuleFor(o => o.id, f => Guid.NewGuid().ToString())
    .RuleFor(o => o.vin, f => f.PickRandom(vinList))
    .RuleFor(o => o.EventName, f => f.PickRandom(eventTypes))
    .RuleFor(o => o.VehicleType, f => f.Vehicle.Type())
    .RuleFor(o => o.FuelType, f => f.Vehicle.Fuel())
    .RuleFor(o => o.timestamp, f => DateTime.Now) // just today's date
    .RuleFor(o => o.date, (f, m) => $"{m.timestamp.ToString("yyyy-MM-dd")}")
    .RuleFor(o => o.s1, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.s2, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.s3, f => f.Random.Double(1, 2000))
    .RuleFor(o => o.region, f => f.Address.Country());
    
    var events = telemetryEvent.Generate(numberOfEventsToGenerate);
       
    return events;

}


Next, let's generate 100 events and write them to our container.

In [ ]:
var telemetryEvents = GenerateTelemetryEvents(100);

foreach (TelemetryEvent telemetryEvent in telemetryEvents)
{
    var itemResponse = await container.CreateItemAsync<TelemetryEvent>(telemetryEvent, new PartitionKey(telemetryEvent.Vin));
}

Display.AsMarkdown(@"
Created 100 items in VehicleData container. 
");
                   

### Query for items
We can run a [SQL query](https://docs.microsoft.com/azure/cosmos-db/sql-query-getting-started) against our container. Azure Cosmos DB returns results in pages, so we use the pattern of calling the [ReadNextAsync](https://docs.microsoft.com/dotnet/api/microsoft.azure.cosmos.feediterator.readnextasync?view=azure-dotnet) method on a [FeedIterator](https://docs.microsoft.com/dotnet/api/microsoft.azure.cosmos.container.getitemqueryiterator?view=azure-dotnet#Microsoft_Azure_Cosmos_Container_GetItemQueryIterator__1_Microsoft_Azure_Cosmos_QueryDefinition_System_String_Microsoft_Azure_Cosmos_QueryRequestOptions_). 

The query ``SELECT * FROM c WHERE c.Vin = '12345'`` will return all documents for the specified ``Vin``. Putting the partition key of ``Vin`` in the WHERE clause ensures that we can run an efficient query. 

In [ ]:
QueryDefinition queryDefinition = new QueryDefinition("SELECT * FROM c WHERE c.Vin = @vin")
                                      .WithParameter("@vin", "12345");

FeedIterator<TelemetryEvent> queryResultSetIterator = container.GetItemQueryIterator<TelemetryEvent>(queryDefinition);

List<TelemetryEvent> telemetryEvents = new List<TelemetryEvent>();

while (queryResultSetIterator.HasMoreResults)
{
    FeedResponse<TelemetryEvent> currentResultSet = await queryResultSetIterator.ReadNextAsync();
    foreach (TelemetryEvent telemetryEvent in currentResultSet)
    {
        telemetryEvents.Add(telemetryEvent);
    }
}

telemetryEvents

### Visualize our data
TODO: Come up with nteract visualization once that is working

### Next steps
- See [samples](https://github.com/Azure/azure-cosmos-dotnet-v3/tree/master/Microsoft.Azure.Cosmos.Samples) for the [Azure Cosmos DB .NET SDK](https://www.nuget.org/packages/Microsoft.Azure.Cosmos/)
- Learn about how to choose the right [throughput (RU/s)](https://docs.microsoft.com/azure/cosmos-db/estimate-ru-with-capacity-planner) for a workload
- Learn about how to [choose a good partition key](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview) and [model your data](https://docs.microsoft.com/azure/cosmos-db/how-to-model-partition-example)